<a href="https://colab.research.google.com/github/shaoncsecu/BN-DRISHTI/blob/main/test_scripts/BN_DRISHTI_Run_On_Test_Sets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##YoloV5 Setup...

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

###Downloading Models.

In [ ]:
import os
path14 = "/content/model"
if os.path.exists(path14) != True:
  '''Downloading the model into CoLab temporary directory'''
  !mkdir /content/model
  !wget -P /content/model/ 'https://huggingface.co/crusnic/BN-DRISHTI/resolve/main/models/line_model_best.pt'
  !wget -P /content/model/ 'https://huggingface.co/crusnic/BN-DRISHTI/resolve/main/models/word_model_best.pt'

##Importing necessary libraries...

In [ ]:
from os.path import join
import numpy as np
import operator
import cv2
import fnmatch
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import glob
import imutils
from math import *
from scipy.stats import mode
from distutils.dir_util import copy_tree
from google.colab.patches import cv2_imshow

##Unziping the input Images with Groundtruth...

In [ ]:
def file_unzip(zip_file):
  # Copying the dataset into colab local directory (for faster execution)
  !unzip $zip_file -d "/content"

##Calculating IoU, DR, RA, FM by comparing Sorted line (labels) and GroundTruth

In [ ]:
# Sorting images, labels (ground, predict) on the list sequentially
def sort_label(images, gt_labels, pred_labels):
    sorted_img = []
    sorted_gt_lb = []
    sorted_pred_lb = []
    ids = [img.split('_')[0] for img in images]
    ids.sort(key=int)
    for i in range(len(ids)):
        for ele in images:
            st = ele.split('_')[0]
            if st == ids[i]:
                sorted_img.append(ele)
        for ele in gt_labels:
            st = ele.split('_')[0]
            if st == ids[i]:
                sorted_gt_lb.append(ele)
        for ele in pred_labels:
            st = ele.split('_')[0]
            if st == ids[i]:
                sorted_pred_lb.append(ele)
    return sorted_img, sorted_gt_lb, sorted_pred_lb


# A function to convert the values (x, y, h, w) of Yolo to Pascal Voc
def yolo_to_voc(width, height, x, y, w, h):
    xmax = int((x * width) + (w * width) / 2.0)
    xmin = int((x * width) - (w * width) / 2.0)
    ymax = int((y * height) + (h * height) / 2.0)
    ymin = int((y * height) - (h * height) / 2.0)
    return (xmin, ymin, xmax, ymax)

'''
This function takes the predicted bounding box and ground truth bounding box and
return the IoU ratio
'''
def calc_iou(gt_bbox, pred_bbox):
    #print(gt_bbox)
    x_topleft_gt, y_topleft_gt, x_bottomright_gt, y_bottomright_gt = gt_bbox

    #print(x_topleft_gt, y_topleft_gt, x_bottomright_gt, y_bottomright_gt)

    x_topleft_p, y_topleft_p, x_bottomright_p, y_bottomright_p = pred_bbox

    if (x_topleft_gt > x_bottomright_gt) or (y_topleft_gt > y_bottomright_gt):
        raise AssertionError("Ground Truth Bounding Box is not correct")
    if (x_topleft_p > x_bottomright_p) or (y_topleft_p > y_bottomright_p):
        raise AssertionError("Predicted Bounding Box is not correct", x_topleft_p, x_bottomright_p, y_topleft_p,
                             y_bottomright_gt)

    # if the GT bbox and predcited BBox do not overlap then iou=0
    if (x_bottomright_gt < x_topleft_p):
        # If bottom right of x-coordinate  GT  bbox is less than or above the top left of x coordinate of  the predicted BBox

        return 0.0
    if (
            y_bottomright_gt < y_topleft_p):  # If bottom right of y-coordinate  GT  bbox is less than or above the top left of y coordinate of  the predicted BBox

        return 0.0
    if (
            x_topleft_gt > x_bottomright_p):  # If bottom right of x-coordinate  GT  bbox is greater than or below the bottom right  of x coordinate of  the predcited BBox

        return 0.0
    if (
            y_topleft_gt > y_bottomright_p):  # If bottom right of y-coordinate  GT  bbox is greater than or below the bottom right  of y coordinate of  the predcited BBox

        return 0.0

    GT_bbox_area = (x_bottomright_gt - x_topleft_gt + 1) * (y_bottomright_gt - y_topleft_gt + 1)
    Pred_bbox_area = (x_bottomright_p - x_topleft_p + 1) * (y_bottomright_p - y_topleft_p + 1)

    x_top_left = np.max([x_topleft_gt, x_topleft_p])
    y_top_left = np.max([y_topleft_gt, y_topleft_p])
    x_bottom_right = np.min([x_bottomright_gt, x_bottomright_p])
    y_bottom_right = np.min([y_bottomright_gt, y_bottomright_p])

    intersection_area = (x_bottom_right - x_top_left + 1) * (y_bottom_right - y_top_left + 1)

    union_area = (GT_bbox_area + Pred_bbox_area - intersection_area)

    return intersection_area / union_area

def multiple_image_result(iou_tresh, image_path, images, gt_labels, pred_labels, save_path):

    # Declaring & initializing all necessary variables
    avg_iou = []
    avg_precision = []
    avg_recall = []
    # iou_thr1 = 0.5
    # iou_thr2 = 0.85
    iou_thr2 = iou_tresh
    o2o = []
    N = []
    M = []
    DR = []
    RA = []

    # Taking Single images from image list with for loop Starts Here
    for i in range(len(images)):

        # Taking image_path, groundtruth & prediction label for single image
        img_path = image_path + images[i]
        gt_lb_path = gt_path + gt_labels[i]
        pred_lb_path = pred_path + pred_labels[i]

        # Taking an image dimention (height, width) & Channels type (here, rgb = 3) of single image
        img = cv2.imread(img_path)
        height, width, channels = img.shape

        # Reading labels (groundtruth & prediction) of single image from given path
        fl = open(gt_lb_path, 'r')
        f2 = open(pred_lb_path, 'r')
        gt_lb_data = fl.readlines()
        pred_lb_data = f2.readlines()
        fl.close()
        f2.close()

        # Declaring list to store All lines values from label (groundtruth & prediction) of single image
        all_gt_lb = []
        all_pred_lb = []

        # Storing all values line by line into all_gt_lb list from Groundtruth label of single image
        for dt in gt_lb_data:
            class_id, x, y, w, h = map(float, dt.split(' '))
            new_label = list(yolo_to_voc(width, height, x, y, w, h))
            all_gt_lb.append(new_label)

        # Storing all values line by line into all_pred_lb list from Prediction label of single image
        for dt in pred_lb_data:
            class_id, x, y, w, h, conf = map(float, dt.split(' '))
            new_label = list(yolo_to_voc(width, height, x, y, w, h))
            all_pred_lb.append(new_label)

        # Declaring a list to store IoU's of a single image
        ious = []
        '''
        Doing comparison with a prediction line value to all groundtruth line value one by one. 
        So that we can take the closest values and take the IoU for each prediction line and store it in ious list. 
        '''
        for ipb, pred_box in enumerate(all_pred_lb):
            for igb, gt_box in enumerate(all_gt_lb):
                iou = calc_iou(gt_box, pred_box)
                temp = iou_thr2
                if iou > iou_thr2:
                    ious.append(iou)

        '''
        Here we are taking o2o = one to one matchs and N = Number of lines in the groundtruth, 
        M = Number of lines in the Prediction... per image. Then calculating DR & RA per image.
        '''
        o2o_per_img = len(ious)
        N_per_img = len(all_gt_lb)
        M_per_img = len(all_pred_lb)

        o2o.append(o2o_per_img)
        N.append(N_per_img)
        M.append(M_per_img)


        '''
        As we have a list of IoU per image in ious, so here we are adding all the IoU's in for loop.
        Then dividing it with list length. As a result we are getting append IoU for single image in avg_iou in one iteration
        and eventually at the end of the loop all the IoU's for every image in the directory will be in avg_iou list... 
        '''
        # per_img_iou = 0
        # for i in range(len(ious)):
        #     per_img_iou += ious[i]
        # avg_iou.append(per_img_iou/len(ious))

        per_img_iou = 0
        if N_per_img != 0:
            per_line_iou = 0
            for i in range(len(ious)):
                per_line_iou += ious[i]
            per_img_iou = per_line_iou / N_per_img
            avg_iou.append(per_img_iou)
        else:
            avg_iou.append(per_img_iou)




    # Taking Single images from image list with for loop Ends Here

    '''
    As we are out of the loop, now we have IoU for every image in avg_iou list. 
    Here we are summing all the IoU's in avg_sum_iou...
    '''
    avg_sum_iou = 0
    for i in range(len(avg_iou)):
        avg_sum_iou += avg_iou[i]

    # In order to show IoU's of corresponding image together, we are zipping it and printing it out...
    print("Images with IoU's :::")
    for img, iou in zip(images, avg_iou):
        print(img, iou)
    
    # save_path = '/content/Final_Results/'
    # file_name = "Results_o2o_N_M_IoU_DR_RA_FM.txt"
    file_name = "Results_o2o_N_M_IoU_DR_RA_FM_@"+str(iou_thr2)+".txt"
    completeName = os.path.join(save_path, file_name)
    f = open(completeName, "w+")

    dr = 0
    ra = 0
    fm = 0

    f.write("Given Treshold: %f\n" %temp)
    f.write("Images - o2o - N - M - IoU - DR - RA - FM :::\n")
    print("Images-o2o-N-M-IoU-DR-RA-FM :::")
    for img, o2, n, m, iou in zip(images, o2o, N, M, avg_iou):
        dr = o2 / n
        ra = o2 / m
        fm = (2 * dr * ra) / (dr + ra)
        print(img, o2, n, m, iou, dr, ra, fm)
        f.write("%s, %i, %i, %i, %.2f, %.2f, %.2f, %.2f\n" % (img, o2, n, m, iou, dr, ra, fm))
        f.write("\n")


    # Calculating final IoU By dividing avg_sum_iou by length of list length of avg_iou which is 150
    Final_IoU = avg_sum_iou/(len(avg_iou))
    print("Average IoU for 150 images (Final IoU of Dataset) :: {}".format(Final_IoU))

    # o2o Starts
    Final_o2o = 0
    for j in range(len(o2o)) :
        Final_o2o += o2o[j]
    print(Final_o2o)

    # N Starts
    Final_N = 0
    for j in range(len(N)):
        Final_N += N[j]
    print(Final_N)

    # M Starts
    Final_M = 0
    for j in range(len(M)):
        Final_M += M[j]
    print(Final_M)

    f.write("\nFinal (IoU - o2o - N - M) :::\n")
    f.write("%f, %s, %s, %s\n" % (Final_IoU, Final_o2o, Final_N, Final_M))
    f.write("\n")

    DR = Final_o2o/Final_N
    RA = Final_o2o/Final_M

    print("Detection Rate(Recall) :: {}".format(DR))
    print("Recognition Rate(Precision) :: {}".format(RA))

    # Calculating final performance and printing it out...
    FM = ((2 * DR) * RA) / (DR + RA)
    print("Final Performance :: {}".format(FM*100))

    f.write("\nDR - RA - FM :::\n")
    f.write("%f, %f, %f\n" % (DR, RA, FM))
    f.write("\n")

    f.close()
    f


In [ ]:
def cal_iou_dr_ra_fm(iou_tresh, img_path, gt_path, pred_path, save_path):
  images = os.listdir(img_path)
  gt_labels = os.listdir(gt_path)
  pred_labels = os.listdir(pred_path)

  # Calling sort_label() and giving images, groundtruth and predict label to sort them sequentially
  images, gt_labels, pred_labels = sort_label(images, gt_labels, pred_labels)

  print("Sorted Lists :::")
  print("Images: ",images)
  print("GroundTruth: ",gt_labels)
  print("Prediction: ",pred_labels) 

  if (not gt_path):
    print("You haven't given the GroundTruths of your Test Images!")
  else:
    # Calling out multiple_image_result() to get the average IoUs, DR, RA of dataset...
    multiple_image_result(iou_tresh, img_path, images, gt_labels, pred_labels, save_path)

#**Detecting lines with YOLO**

##**1st Detection: Detecting the Input Images** (Upload the given trained Line Model weights on your drive and assign its directory to the --weights command.).

Defining a function where we pass necessary attributes to detect lines by YOLO.

In [ ]:
# Yolo Detection...
def yolo_detection(img_path, img_size, conf):
  # %cd yolov5
  !python /content/yolov5/detect.py --weights /content/model/line_model_best.pt --img $img_size --conf $conf --source $img_path --save-conf --save-txt

##Sorted line (labels) after 1st detection

In [ ]:
class Line_sort:
    def __init__(self, txt_files, txt_loc):
        self.txt_files = txt_files
        self.txt_loc = txt_loc
        self.read_file()

    def read_file(self):
        files = self.txt_files
        os.mkdir('/content/sorted_line_after_1st_detection')
        for file in files:
            txt_file = []
            file_loc = self.txt_loc+file
            with open(file_loc, 'r' , encoding='utf-8',errors='ignore') as lines:
                for line in lines:
                    print(line)
                    token = line.split()
                    print(token)
                    _, x, y, w, h, conf = map(float, line.split(' '))
                    print("width -> ",w)
                    print("confidence -> ",conf)
                    if w > 0.50 and conf < 0.50:
                      continue
                    else:
                      txt_file.append(token)
            sorted_txt_file = sorted(txt_file, key=operator.itemgetter(2))
            # print(sorted_txt_file)
            lenght = len(sorted_txt_file[0])
            self.file_write(sorted_txt_file, file)

    def file_write(self,txt_file, file_name):
        loc = '/content/sorted_line_after_1st_detection/'+file_name
        with open(loc, 'w') as f: 
            c=0
            for line in txt_file:  
                for l in line:
                    c+=1
                    if c == len(line):
                        f.write('%s' % l)
                    else:
                        f.write('%s ' % l)
                f.write("\n")
                c=0


In [ ]:
def sort_detection_label(txt_loc):
  txt_files = os.listdir(txt_loc)
  obj = Line_sort(txt_files,txt_loc)

##1st Line Segment from Sorted line (labels) after 1st detection

In [ ]:
# text file and image sorting
def images_and_txtfile_sort(images, txt_file):
    txt = []
    image = []
    for item in txt_file:
        ch = ""
        for c in item:
            if c == "_":
                break
            ch += c
        txt.append(ch)

    txt.sort(key=int)
    sorted_txtfiles = []
    sorted_images = []

    for i in range(len(txt)):
        for ele, ele2 in zip(txt_file, images):
            st = ""
            st2 = ""
            for ch in ele:
                if ch == "_":
                    break
                st += ch
            if st == txt[i]:
                sorted_txtfiles.append(ele)
            for ch in ele2:
                if ch == "_":
                    break
                st2 += ch
            if st2 == txt[i]:
                sorted_images.append(ele2)
    return sorted_images, sorted_txtfiles

# line segmantation by yolo bounding box
def line_segmantation(path, image_loc, txt_loc, images, txt_file):

    image, txt_files = images_and_txtfile_sort(images, txt_file)

    for image, txt in zip(image, txt_files):
        # make folder name according to txt file
        new_folder = txt[:-4]
        new_folder_loc = path + new_folder
        # create subfolder in Lines according to txt file
        os.mkdir(new_folder_loc)
        # current image location
        current_image = image_loc + image
        img = cv2.imread(current_image)
        dh, dw, _ = img.shape
        # current txt file location
        current_txt = txt_loc + txt
        fl = open(current_txt, 'r')
        data = fl.readlines()
        fl.close()
        k=1
        for dt in data:
            # _, x, y, w, h = map(float, dt.split(' '))
            _, x, y, w, h,  conf = map(float, dt.split(' '))
            if w > 0.50 and w < 0.80:
               x = 0.5
               w = 1.0
            l = int((x - w / 2) * dw)
            r = int((x + w / 2) * dw)
            t = int((y - h / 2) * dh)
            b = int((y + h / 2) * dh)

            crop = img[t:b, l:r]
            cv2.imwrite("{}{}/{}_{}.jpg".format(path, new_folder, new_folder, k), crop)
            k += 1

def take_valid_img(images):
    image = []
    valid_img_ext = ["jpg", "JPG", "jpeg", "JPEG", "png", "PNG"]
    for img in images:
        try:
            ext = img.split('.')[1]
            if ext not in valid_img_ext :
                continue
            else:
                image.append(img)
        except:
            continue
    return image


In [ ]:
def line_segmentation(img_path, sorted_label):
  current_directory = img_path
  #print(current_directory)
  current_directory1 = sorted_label
  #print(current_directory1)
  images = []  # take for all images
  txt = []     # take for all txt files

  # images
  # take all files in list in current directory location
  current_directory_files = os.listdir(current_directory) 
  #print(current_directory_files)
  if not current_directory_files:
      print("{} folder is empty!")

  # lines
  # take all files in list in current directory1 location
  current_directory_1_files = os.listdir(current_directory1)
  if not current_directory_1_files:
      print("Line folder is empty of {} !")

  txt_pattern = "*.txt"
  # take all only image in images list from current directory files list
  images1 = [entry for entry in current_directory_files]
  images = take_valid_img(images1)

  # take all only txt file in txt list from current directory1 files list
  txt = [entry for entry in current_directory_1_files if fnmatch.fnmatch(entry, txt_pattern)]

  os.mkdir('/content/croped_line_after_1st_detection')
  path = "/content/croped_line_after_1st_detection/"
  # calling the line segmantation function
  line_segmantation(path, current_directory, current_directory1, images, txt)
  print("Successful line segmantated in the {} folder".format(path))

#Rotation

DSkew by HaughLine...

In [ ]:
class ImgCorrect():
    def __init__(self, img):
        self.img = img
        self.h, self.w, self.channel = self.img.shape
        if self.w <= self.h:
            self.scale = 700 / self.w
            self.w_scale = 700
            self.h_scale = self.h * self.scale
            self.img = cv2.resize(self.img, (0, 0), fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        else:
            self.scale = 700 / self.h
            self.h_scale = 700
            self.w_scale = self.w * self.scale
            self.img = cv2.resize(self.img, (0, 0), fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        self.gray = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY)

    def img_lines(self):
        ret, binary = cv2.threshold(self.gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
        # cv2.imshow("bin",binary)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))  # rectangular structure
        binary = cv2.dilate(binary, kernel)  # dilate
        edges = cv2.Canny(binary, 50, 200)
        # cv2.imshow("edges", edges)
        self.lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=100, maxLineGap=20)
        print(self.lines)
        if self.lines is None:
            print("Line segment not found")
            return None

        lines1 = self.lines[:, 0, :]  # Extract as 2D
        # print(lines1)
        imglines = self.img.copy()
        for x1, y1, x2, y2 in lines1[:]:
            cv2.line(imglines, (x1, y1), (x2, y2), (0, 255, 0), 3)
        return imglines

    def search_lines(self):
      lines = self.lines[:, 0, :]  # extract as 2D
    
      number_inexist_k = 0
      sum_pos_k45 = number_pos_k45 = 0
      sum_pos_k90 = number_pos_k90 = 0
      sum_neg_k45 = number_neg_k45 = 0
      sum_neg_k90 = number_neg_k90 = 0
      sum_zero_k = number_zero_k = 0

      for x in lines:
          if x[2] == x[0]:
              number_inexist_k += 1
              continue
          #print(degrees(atan((x[3] - x[1]) / (x[2] - x[0]))), "pos:", x[0], x[1], x[2], x[3], "Slope:",(x[3] - x[1]) / (x[2] - x[0]))
          degree = degrees(atan((x[3] - x[1]) / (x[2] - x[0])))
          #print(degree)
          if 0 < degree < 45:
              number_pos_k45 += 1
              sum_pos_k45 += degree
          if 45 <= degree < 90:
              number_pos_k90 += 1
              sum_pos_k90 += degree
          if -45 < degree < 0:
              number_neg_k45 += 1
              sum_neg_k45 += degree
          if -90 < degree <= -45:
              number_neg_k90 += 1
              sum_neg_k90 += degree
          if x[3] == x[1]:
              number_zero_k += 1

      max_number = max(number_inexist_k, number_pos_k45, number_pos_k90, number_neg_k45,number_neg_k90, number_zero_k)
      #print(number_inexist_k,number_pos_k45, number_pos_k90, number_neg_k45, number_neg_k90,number_zero_k)
    
      if max_number == number_inexist_k:
          return 90
      if max_number == number_pos_k45:
          return sum_pos_k45 / number_pos_k45
      if max_number == number_pos_k90:
          return sum_pos_k90 / number_pos_k90
      if max_number == number_neg_k45:
          return sum_neg_k45 / number_neg_k45
      if max_number == number_neg_k90:
          return sum_neg_k90 / number_neg_k90
      if max_number == number_zero_k:
          return 0

    def rotate_image(self, degree):
        """
                 Positive angle counterclockwise rotation
        :param degree:
        :return:
        """
        print("degree:", degree)
        if -45 <= degree <= 0:
            degree = degree  # #negative angle clockwise
        if -90 <= degree < -45:
            degree = 90 + degree  # positive angle counterclockwise
        if 0 < degree <= 45:
            degree = degree  # positive angle counterclockwise
        if 45 < degree <= 90:
            degree = degree - 90  # negative angle clockwise
        print("rotate degree:", degree)

        # degree = degree - 90
        height, width = self.img.shape[:2]
        heightNew = int(width * fabs(sin(radians(degree))) + height * fabs(
            cos(radians(degree))))  # This formula refers to the previous content
        widthNew = int(height * fabs(sin(radians(degree))) + width * fabs(cos(radians(degree))))

        matRotation = cv2.getRotationMatrix2D((width / 2, height / 2), degree, 1)  # rotate degree counterclockwise
        matRotation[0, 2] += (widthNew - width) / 2
        # Because after rotation, the origin of the coordinate system is the upper left corner of the new image, so it needs to be converted according to the original image
        matRotation[1, 2] += (heightNew - height) / 2

        # Affine transformation, the background color is filled with white
        imgRotation = cv2.warpAffine(self.img, matRotation, (widthNew, heightNew), borderValue=(255, 255, 255))
        # imgRotation = cv2.warpAffine(self.img, matRotation, (widthNew, heightNew), borderValue=filled_color)

        return imgRotation

def dskew(line_path, img):
    img_loc = line_path + img
    im = cv2.imread(img_loc)

    # Padding
    bg_color = [255, 255, 255]
    pad_img = cv2.copyMakeBorder(im,100,100,100,100,cv2.BORDER_CONSTANT,value=bg_color)

    imgcorrect = ImgCorrect(pad_img)
    lines_img = imgcorrect.img_lines()
    print(type(lines_img))
    if lines_img is None:
        rotate = imgcorrect.rotate_image(0)
    else:
        degree = imgcorrect.search_lines()
        rotate = imgcorrect.rotate_image(degree)

    return rotate




HaughLine and Affine Transform...

In [ ]:
rotate_line = "/content/Rotated_line_by_HaughLine_Affine/"
os.mkdir(rotate_line)

rotate_line_Dskew = "/content/Final_Results/DSkew/"
# os.mkdir(rotate_line_Dskew)

rotate_line_Haughline = "/content/Final_Results/HaughLine_Affine/"
# os.mkdir(rotate_line_Haughline)

exception = []

# Degree conversion
def DegreeTrans(theta):
    res = theta / np.pi * 180
    return res

# Rotate the image degree counterclockwise (original size)
def rotateImage(src, degree):
    # The center of rotation is the center of the image
    h, w = src.shape[:2]
    # Calculate the two-dimensional rotating affine transformation matrix
    RotateMatrix = cv2.getRotationMatrix2D((w / 2.0, h / 2.0), degree, 1)
    #print(RotateMatrix)
    # Affine transformation, the background color is filled with white
    rotate = cv2.warpAffine(src, RotateMatrix, (w, h), borderValue=(255, 255, 255))

    # Padding
    bg_color = [255, 255, 255]
    pad_image_rotate = cv2.copyMakeBorder(rotate,100,100,100,100,cv2.BORDER_CONSTANT,value=bg_color)

    return pad_image_rotate

# Calculate angle by Hough transform
def CalcDegree(srcImage,canny_img):
    lineimage = srcImage.copy()
    # Detect straight lines by Hough transform
    # The fourth parameter is the threshold, the greater the threshold, the higher the detection accuracy
    try:
        lines = cv2.HoughLines(canny_img, 1, np.pi / 180, 200)
        # Due to different images, the threshold is not easy to set, because the threshold is set too high, so that the line cannot be detected, the threshold is too low, the line is too much, the speed is very slow
        sum = 0
        # Draw each line segment in turn
        for i in range(len(lines)):
            for rho, theta in lines[i]:
                # print("theta:", theta, " rho:", rho)
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a * rho
                y0 = b * rho
                x1 = int(round(x0 + 1000 * (-b)))
                y1 = int(round(y0 + 1000 * a))
                x2 = int(round(x0 - 1000 * (-b)))
                y2 = int(round(y0 - 1000 * a))
                # Only select the smallest angle as the rotation angle
                sum += theta
                cv2.line(lineimage, (x1, y1), (x2, y2), (0, 0, 255), 1, cv2.LINE_AA)
        average = sum / len(lines)
        angle = DegreeTrans(average) - 90
        return angle
    except:
        angle = 0.0
        return angle

def ready_for_rotate(line_path, img):
    rotate_line = "/content/Rotated_line_by_HaughLine_Affine/"

    img_loc = line_path + img
    image = cv2.imread(img_loc)

    org_width = image.shape[1]
    org_height = image.shape[0]

    img1 = image
    im_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

    edges = cv2.Canny(im_gray,50,150,apertureSize=3)
    degree = CalcDegree(image,edges)
    #print("Adjust angle:", degree)
    if degree == 0.0:
        rotate = dskew(line_path, img)
        
        filename1 = rotate_line_Dskew + img
        cv2.imwrite(filename1, rotate)
        # filename = rotate_line_exception + img
        filename = rotate_line + img
        exception.append(img)
        cv2.imwrite(filename, rotate)
    else:
        rotate = rotateImage(image, degree)

        filename2 = rotate_line_Haughline + img
        cv2.imwrite(filename2, rotate)

        filename = rotate_line + img
        cv2.imwrite(filename, rotate)



In [ ]:
def rotate_line(first_detection):
  source = first_detection
  dir = os.listdir(source)

  for i in range(len(dir)):
      line_path = source + dir[i] + "/"
      line_dir = os.listdir(line_path)

      for img in line_dir:
          ready_for_rotate(line_path, img)

  # Line Rotate by -90
  os.chdir("/content/Rotated_line_by_HaughLine_Affine/")
  for file in glob.glob("*.jpg"):
      image = Image.open(file)
      width, height = image.size
      if height > width:
          angle = -90
          rotated = image.rotate(angle, expand=True)
          rotated.save(file.replace(".jpg", ".jpg"))

Copy Directory content...

In [ ]:
def copy_dir(from_dir, to_dir):
  # copy subdirectory example
  fromDirectory = from_dir
  toDirectory = to_dir
  os.mkdir(toDirectory)

  copy_tree(fromDirectory, toDirectory)

#**Final Line Segmentation (**Upload the given trained Line Model weights on your drive and assign its directory to the --weights command.**)**

**However, assign the directory of the best weights of your YOLO Line model in --weights directory.**

In [ ]:
def yolo_detection2(img_path, img_size, conf):
  !python /content/yolov5/detect.py --weights /content/model/line_model_best.pt --img $img_size --conf $conf --source $img_path --save-conf --save-txt

##Find undetected image in Yolo 2nd detection...

Defining a function to get those images that are not getting 2nd YOLO detection for their low dimension and putting those images with the final segmented images.  

In [ ]:
def find_undetected_images():
  img_path = "/content/Rotated_line_by_HaughLine_Affine/"
  # detect_lb_path = "/content/Rotated_line_by_HaughLine_Affine/runs/detect/exp/labels/"
  detect_lb_path = "/content/yolov5/runs/detect/exp2/labels/"
  # os.mkdir("undetect_img")
  undetect_img_path = "/content/Final_Results/Final_Line_Segmentation/"
  os.mkdir(undetect_img_path)

  def take_valid_img(images):
      image = []
      valid_img_ext = ["jpg", "JPG", "jpeg", "JPEG", "png", "PNG"]
      for img in images:
          try:
              ext = img.split('.')[1]
              if ext not in valid_img_ext :
                  continue
              else:
                  image.append(img)
          except:
              continue
      return image

  img1 = os.listdir(img_path)
  img = take_valid_img(img1)
  detect_lb = os.listdir(detect_lb_path)

  def find_undetect_img(img,detect_lb):
      img_lb = [im.split('.')[0] for im in img]
      dt_lb = [dt.split('.')[0] for dt in detect_lb]
      undt_lb = list(set(img_lb).difference(dt_lb))
      undetect_img = []
      detect_img = []
      for lb in undt_lb:
          for im in img:
              im_lb = im.split('.')[0]
              if lb == im_lb:
                  undetect_img.append(im)
              else:
                  detect_img.append(im)
      write_image(undetect_img)

  def write_image(undt_img):
      for im in undt_img:
          filename = undetect_img_path+im
          img = cv2.imread(img_path+im)
          cv2.imwrite(filename,img)

  find_undetect_img(img,detect_lb)





## 2nd Line Segmentation after Rotation...

Cropping line from Sorted line (labels) after 2nd detection

In [ ]:
def final_line_segmentation():
  filename2 = "/content/Final_Results/Final_Line_Segmentation/"
  list2 = os.listdir(filename2)

  filename1 = "/content/Rotated_line_by_HaughLine_Affine/"
  list1 = os.listdir(filename1)
  # list1.remove("runs")

  undetected_img = list2
  destination = filename2

  # label_path = "/content/Rotated_line_by_HaughLine_Affine/runs/detect/exp/"
  label_path = "/content/yolov5/runs/detect/exp2/"

  exception = []
  def arrange_images_labels(images, labels):
      img_lb = {}
      for img in images:
          for lb in labels:
              st_img = img.split('.')[0]
              st_lb = lb.split('.')[0]
              if st_img == st_lb:
                  img_lb[img] = lb
      return img_lb

  def take_valid_img(images, undetected_img):
      image = []
      valid_img_ext = ["jpg", "JPG", "jpeg", "JPEG", "png", "PNG"]
      for img in images:
          try:
              ext = img.split('.')[1]
              if ext not in valid_img_ext :
                  continue
              else:
                  image.append(img)
          except:
              continue
      detected_img_val = [x for x in image if x not in undetected_img]
      return detected_img_val


  def crop_image(bb_data,destination,image,img_lb,dh,dw):
      x = float(bb_data[1])
      y = float(bb_data[2])
      w = float(bb_data[3])
      h = float(bb_data[4])
    
      # x = 0.5
      # w  = 1.0
      l = int((x - w / 2) * dw)
      r = int((x + w / 2) * dw)
      t = int((y - h / 2) * dh)
      b = int((y + h / 2) * dh)

      crop = image[t:b, l:r]
      filename = destination+img_lb
      cv2.imwrite(filename,crop)
      
    
  # line segmantation by yolo bounding box
  def line_segmantation(image_loc, txt_loc, images, txt_file, destination):
      image_labels = arrange_images_labels(images, txt_file)

      for img_lb, lb in image_labels.items():
          img_name = lb[:-4]
          current_image = image_loc + img_lb
          image = cv2.imread(current_image)
          dh, dw, _ = image.shape
          current_txt = txt_loc + lb
          fl = open(current_txt, 'r')
          data = fl.readlines()
          fl.close()
          
          max_w = 0
          data1 = []
          for line in data:
              token = line.split()
              data1.append(token)
          
          if len(data1)==1:
              bb_data = data1[0]
              wdth = float(bb_data[3])
              if wdth>0.4:
                  crop_image(bb_data,destination,image,img_lb,dh,dw,)
              else:
                  filename = destination+img_lb
                  cv2.imwrite(filename,image)
          elif len(data1)==2:
              bb_data1 = data1[0]
              bb_data2 = data1[1]
              w1 = float(bb_data1[3]) 
              w2 = float(bb_data2[3])
              # if w1 <= 0.5 and w2 <= 0.5:
              #    max_w = max_w + 1
              # if max_w == 2:
              #   filename = destination+img_lb
              #   cv2.imwrite(filename,image)
              c1 = float(bb_data1[5]) 
              c2 = float(bb_data2[5])
              if w1 <= 0.5 and w2 <= 0.5:
                if c1 >= 0.8 and c2 >= 0.8:
                  sorted_bb_data = sorted(data1, key=operator.itemgetter(5))
                  bb_data = sorted_bb_data[-1]
                  crop_image(bb_data,destination,image,img_lb,dh,dw,)
                else:
                  filename = destination+img_lb
                  cv2.imwrite(filename,image)
              else:
                sorted_bb_data = sorted(data1, key=operator.itemgetter(3))
                bb_data = sorted_bb_data[-1]
                crop_image(bb_data,destination,image,img_lb,dh,dw,)
          elif len(data1)==3:
              sorted_bb_data = sorted(data1, key=operator.itemgetter(2))
              bb_data = sorted_bb_data[1]
              crop_image(bb_data,destination,image,img_lb,dh,dw,)
          else:
              sorted_bb_data = sorted(data1, key=operator.itemgetter(3))
              bb_data = sorted_bb_data[-1]
              crop_image(bb_data,destination,image,img_lb,dh,dw,)
               
  def read_files(filename1, label_path, undetected_img, destination):
        image_directory = filename1
        txt_label_directory = label_path + "labels/"
        all_img_file = os.listdir(image_directory)

        # take all files in list in current directory1 location
        all_txt_labels = os.listdir(txt_label_directory)

        images = take_valid_img(all_img_file, undetected_img) # image list
        
        txt_pattern = "*.txt"
        all_pred_label = [entry for entry in all_txt_labels if fnmatch.fnmatch(entry, txt_pattern)]

        if len(images) == len(all_pred_label):
          line_segmantation(image_directory, txt_label_directory, images, all_pred_label, destination)
          print("Successful line segmantation!")
        else:
          print("Unsuccessful line segmantation!")     


  read_files(filename1, label_path, undetected_img, destination)     

Drawing Bounding Box in Segmented Lines

In [ ]:
def drawing_bounding_box():
  rotate_line = "/content/Final_Results/Segmented_line_with_bounding_box/"
  os.mkdir(rotate_line)

  def bounding_box(im):
      im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
      ret, thresh = cv2.threshold(im_gray, 127, 255, cv2.THRESH_BINARY_INV)

      edges = cv2.Canny(im_gray, 50, 150, apertureSize=3)
      minLineLength = 1
      maxLineGap = 10
      try:
          lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength, maxLineGap)
          for line in lines:
              for x1, y1, x2, y2 in line:
                  cv2.line(thresh, (x1, y1), (x2, y2), (0), 3)

          kernel = np.ones((3, 3), np.uint8)

          thresh = cv2.dilate(thresh, kernel, iterations=9)
          # thresh = cv2.morphologyEx(thresh,cv2.MORPH_OPEN, kernel)

          contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
          minArea = 900  # min area for to be considered
          for cnt in contours:
              area = cv2.contourArea(cnt)
              if (area > minArea):
                  rect = cv2.minAreaRect(cnt)
                  box = cv2.boxPoints(rect)
                  box = np.int0(box)
                  print(box)
                  cv2.drawContours(im, [box], 0, (0, 0, 255), 3)
          return box, im
      except:
          box = [[]]
          return box, im



  def draw_bb(line_path, img):
      img_loc = line_path + img
      image = cv2.imread(img_loc)

      box, bb_image = bounding_box(image)

      filename = rotate_line + img
      cv2.imwrite(filename, bb_image)

  line_path = "/content/Final_Results/Final_Line_Segmentation/"
  line_dir = os.listdir(line_path)

  for img in line_dir:
      draw_bb(line_path, img)

Word Cropping of Segmented Lines...

In [ ]:
def word_cropping():
  word = "/content/Final_Results/crop_word_of_segmented_lines/"
  os.mkdir(word)
  img_path = "/content/Final_Results/Final_Line_Segmentation/"
  images = os.listdir(img_path)


  def bounding_box(im):
      im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
      ret, thresh = cv2.threshold(im_gray, 127, 255, cv2.THRESH_BINARY_INV)
      edges = cv2.Canny(im_gray, 50, 150, apertureSize=3)

      minLineLength = 1
      maxLineGap = 10
      word_box = []
      try:
          lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength, maxLineGap)
          for line in lines:
              for x1, y1, x2, y2 in line:
                  cv2.line(thresh, (x1, y1), (x2, y2), (0), 3)

          kernel = np.ones((3, 3), np.uint8)
          dilate = cv2.dilate(thresh, kernel, iterations=9)

          contours, hierarchy = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
          minArea = 900  # min area for to be considered
          for cnt in contours:
              area = cv2.contourArea(cnt)
              if (area > minArea):
                  rect = cv2.minAreaRect(cnt)
                  box = cv2.boxPoints(rect)
                  box_rect = []
                  x, y, w, h = cv2.boundingRect(box)
                  box_rect.append(x)
                  box_rect.append(y)
                  box_rect.append(w)
                  box_rect.append(h)
                  word_box.append(box_rect)
          return word_box
      except:
          return word_box

  def word_crop(image,name,word_box):
      sorted_word_box = sorted(word_box, key=operator.itemgetter(0))
      dh,dw,_ = image.shape
      k=1
      im = image.copy()
      for rect in sorted_word_box:
          x = rect[0]
          y = rect[1]
          w = rect[2]
          h = rect[3]
          crop = im[y:y + h, x:x + w]
          filename = word + name + "/" + name +"_"+str(k)+".jpg"
          cv2.imwrite(filename, crop)
          k+=1

  def read_data(images):
      for img in images:
          name = img.split('.')[0]
          folder = word+name
          os.mkdir(folder)
          im_path = img_path+img
          im= cv2.imread(im_path)
          bg = [255, 255, 255]
          pad_img = cv2.copyMakeBorder(im.copy(), 100, 100, 100, 100, cv2.BORDER_CONSTANT, value=bg)
          word_box = bounding_box(pad_img)
          word_crop(pad_img,name,word_box)
          print(img+" -> word cropping is successful for line image!!")

  read_data(images)

Zipping and Downloading Results...

In [ ]:
def zip_results():
  os.mkdir("/content/test_results")

  # Zipping the test Results
  !zip -r /content/test_results/Final_Results.zip /content/Final_Results


##**Main function call starts from here.** (Upload the given BN-HTRd datasets zip version on your desire location, and assign the directory address of the dataset's zip version to the defined "dataset_directory" variable)

In [ ]:
# Downloading the dataset zip from the given directory...
dataset_directory = 'https://github.com/crusnic-corp/BN-DRISHTI/raw/main/test_scripts/test.zip'
!wget -P /content/ $dataset_directory

x = os.listdir("/content")
for i in x:
  if i.endswith(('.zip')) == True:
    zip_file = "/content/" + i
    file_unzip(zip_file) # Unzipping the Zip file...
    !rm -rf $zip_file # Removing downloaded zip file...
    zip_lbl = i.split('.')[0] # Taking label of the dataset...
    # Asigning image and label path according to our dataset structure...
    img_path = "/content/" + zip_lbl + "/images"
    gt_path = "/content/" + zip_lbl + "/labels"

In [ ]:
# Making a folder to store all the results...
os.mkdir("/content/Final_Results/")

# Image Path from unziped file...
# Yolo 1st Detection...
img_path = img_path + "/"
img_size = 640
conf = 0.30
yolo_detection(img_path, img_size, conf)

# Sorting Labels of 1st detection on the basis of y...
txt_loc = "/content/yolov5/runs/detect/exp/labels/"
sort_detection_label(txt_loc)

# Line Segmentation after 1st Detection...
sorted_label = "/content/sorted_line_after_1st_detection"
sorted_label = sorted_label + "/"
line_segmentation(img_path, sorted_label)

#GroundTruth Path from unziped file......
save_path = '/content/Final_Results/Line_Evaluation/'
os.mkdir(save_path)
iou_tresh_range = [0.7, 0.8, 0.85, 0.9]
gt_path = gt_path + "/"
# IoU FM DR RA Calculation...
# pred_path = "/content/yolov5/runs/detect/exp/labels/"
pred_path = "/content/sorted_line_after_1st_detection/"
for i in iou_tresh_range:
  iou_tresh = i
  print("Given IoU: ",iou_tresh)
  cal_iou_dr_ra_fm(iou_tresh, img_path, gt_path, pred_path, save_path)
  print()

# Only YOLO and without Filtering...
# GroundTruth Path from unziped file......
save_path = '/content/Final_Results/Line_Evaluation_only_YOLO/'
os.mkdir(save_path)
iou_tresh_range = [0.7, 0.8, 0.85, 0.9]
gt_path = gt_path + "/"
# IoU FM DR RA Calculation...
pred_path = "/content/yolov5/runs/detect/exp/labels/"
# pred_path = "/content/sorted_line_after_1st_detection/"
for i in iou_tresh_range:
  iou_tresh = i
  print("Given IoU: ",iou_tresh)
  cal_iou_dr_ra_fm(iou_tresh, img_path, gt_path, pred_path, save_path)
  print()


In [ ]:
# Final prediction by YOLO...

def draw_BB(img_path, label_path, flag):
  img = cv2.imread(img_path)
  dh, dw, _ = img.shape

  lb = open(label_path, 'r')
  data = lb.readlines()
  lb.close()

  for dt in data:
    if flag == 0:
      _, x, y, w, h = map(float, dt.split(' '))
    else:
      _, x, y, w, h, conf = map(float, dt.split(' '))
    l = int((x - w / 2) * dw)
    r = int((x + w / 2) * dw)
    t = int((y - h / 2) * dh)
    b = int((y + h / 2) * dh)
    if flag == 0:
      cv2.rectangle(img, (l, t), (r, b), (0, 250, 0), 2)
    else:
      cv2.rectangle(img, (l, t), (r, b), (250, 0, 0), 2)
    
  # plot_fig(img)
  return img


# For Predicted document image Red BB...
# img_path = "/content/offline_line_word_trainning/line_tranning/dataset_for_line_segmentation/test/images/"
img_path = "/content/test/images/"
lb_path = "/content/sorted_line_after_1st_detection/"
des = "/content/Final_Results/Final_doc_image_detection/"
os.mkdir(des)
img_list = os.listdir(img_path)
flag = 1
for i in img_list:
  image_path = img_path + i
  label_path = lb_path + i.split('.')[0] + ".txt"
  img_p = draw_BB(image_path, label_path, flag)
  img_p1 = cv2.cvtColor(img_p, cv2.COLOR_BGR2RGB)
  filename = des + i
  cv2.imwrite(filename, img_p1)

# For Ground truth document image Green BB...
img_path_1 = "/content/test/images/"
lb_path_1 = "/content/test/labels/"
des_1 = "/content/Final_Results/Doc_image_BB/"
os.mkdir(des_1)
img_list_1 = os.listdir(img_path_1)
flag = 0
for i in img_list_1:
  image_path_gt = img_path_1 + i
  label_path_gt = lb_path_1 + i.split('.')[0] + ".txt"
  img_p_gt = draw_BB(image_path_gt, label_path_gt, flag)
  img_p1_gt = cv2.cvtColor(img_p_gt, cv2.COLOR_BGR2RGB)
  filename_1 = des_1 + i
  cv2.imwrite(filename_1, img_p1_gt)

In [ ]:
# Roation..Roatating image after 1st line Segmentation...
rotate_line_Dskew = "/content/Final_Results/DSkew/"
os.mkdir(rotate_line_Dskew)
rotate_line_Haughline = "/content/Final_Results/HaughLine_Affine/"
os.mkdir(rotate_line_Haughline)
first_detection = "/content/croped_line_after_1st_detection/"
rotate_line(first_detection)

# Copy content from one directory to another...
from_dir = "/content/Rotated_line_by_HaughLine_Affine"
to_dir = "/content/Final_Results/Rotated_line_by_HaughLine_Affine"
copy_dir(from_dir, to_dir)

# Image Path from unziped file...
# Yolo 2nd Detection...
rotated_img_path = "/content/Rotated_line_by_HaughLine_Affine/"
# img_size = 416
img_size = 640
conf = 0.50
yolo_detection2(rotated_img_path, img_size, conf)

# FINAL LINE SEGMENTATION :::
# Undetected images from 2nd detection...
find_undetected_images()

In [ ]:
# Crop detected Lines with max Width... 
final_line_segmentation()

In [ ]:
# Taking image back to original shape...
def trim_original_image(rotate, org_w, org_h):
  org_width = org_w
  org_height = org_h

  img1 = rotate
  width = img1.shape[1]
  height = img1.shape[0]
  print("Original height -> ",org_height)
  print("Original width -> ",org_width)

  start_row = 60
  end_row = height - 60

  start_col = 60
  end_col = width - 60
  img_new = img1[start_row:end_row, start_col:end_col]

  width1 = img_new.shape[1]
  height1 = img_new.shape[0]
  print("New height -> ",height1)
  print("New width -> ",width1)
    
  return img_new

def plot_fig(img, size = 15):
  plt.figure(figsize=(size, size))
  plt.imshow(imutils.opencv2matplotlib(img))
  plt.show()

final_line_segment = "/content/Final_Results/Final_Line_Segmentation/"
dskew_img_list = os.listdir(rotate_line_Dskew)
print(dskew_img_list)
for i in dskew_img_list:
  temp = final_line_segment + i
  img1 = cv2.imread(temp)
  height, width, channels = img1.shape
  temp2 = rotate_line_Dskew + i
  img2 = cv2.imread(temp2)
  height2, width2, channels = img2.shape
  if height >= height2:
    print(height2)
    print(width2)
    print(i)
    # os.remove(temp)
    temp3 = trim_original_image(img1, width, height)
    cv2.imwrite(temp, temp3)
    print("Original")
    # cv2_imshow(img1)
    plot_fig(img1)
    print("New")
    # cv2_imshow(temp3)
    plot_fig(temp3)


# **Word Detection and Segmentation** (Upload the given trained Word Model weights on your drive and assign its directory to the --weights directory.).

In [ ]:
# %cd yolov5
!python /content/yolov5/detect.py --weights /content/model/word_model_best.pt --img 640 --conf 0.40 --source /content/Final_Results/Final_Line_Segmentation --save-conf --save-txt

In [ ]:
# Copying directory content...
from_dir = "/content/yolov5/runs/detect/exp3"
to_dir = "/content/Final_Results/Final_word_segmentation_result"
copy_dir(from_dir, to_dir)

Draw Bounding Box

In [ ]:
def draw_BB(img_path, label_path, flag):
  img = cv2.imread(img_path)
  dh, dw, _ = img.shape

  lb = open(label_path, 'r')
  data = lb.readlines()
  lb.close()

  for dt in data:
    if flag == 0:
      _, x, y, w, h = map(float, dt.split(' '))
    else:
      _, x, y, w, h, conf = map(float, dt.split(' '))
    l = int((x - w / 2) * dw)
    r = int((x + w / 2) * dw)
    t = int((y - h / 2) * dh)
    b = int((y + h / 2) * dh)
    if flag == 0:
      cv2.rectangle(img, (l, t), (r, b), (0, 250, 0), 2)
    else:
      cv2.rectangle(img, (l, t), (r, b), (0, 0, 250), 2)
    
  # plot_fig(img)
  return img


Word Segmentation

In [ ]:
class word_segmentation:
    def __init__(self, loc, loc1):
        self.img_loc = loc1
        self.lb_loc = loc+'labels/'
        self.line_img = os.listdir(self.img_loc)
        # self.line_img.remove('labels')
        self.line_lb = os.listdir(self.lb_loc)
        self.segmentation()

    def arrange_img_with_lb(self):
        img_lb = {}
        for img in self.line_img:
            for lb in self.line_lb:
                st_img = img.split('.')[0]
                st_lb = lb.split('.')[0]
                if st_img == st_lb:
                    img_lb[img] = lb
        return img_lb

    def segmentation(self):
        img_with_lb = self.arrange_img_with_lb()
        for image, label in img_with_lb.items():
            new_folder = image.split('.')[0]
            new_folder_loc = self.lb_loc + new_folder
            os.mkdir(new_folder_loc)
            current_img = self.img_loc+image
            img = cv2.imread(current_img)
            dh, dw, _ = img.shape
            current_txt = self.lb_loc + label
            fl = open(current_txt, 'r')
            data = fl.readlines()
            fl.close()
            word_bb = []
            for wd in data:
                token = wd.strip().split()
                word_bb.append(token)
            sorted_word_bb = sorted(word_bb, key=operator.itemgetter(1))
            k = 1

            # Writing Sorted Files...
            self.file_write(sorted_word_bb, label)

            for rect in sorted_word_bb:
                x = float(rect[1])
                y = float(rect[2])
                w = float(rect[3])
                h = float(rect[4])
                # if w > 0.50:
                #   continue
                l = int((x - w / 2) * dw)
                r = int((x + w / 2) * dw)
                t = int((y - h / 2) * dh)
                b = int((y + h / 2) * dh)
                crop = img[t:b, l:r]
                cv2.imwrite("{}{}/{}_{}.jpg".format(self.lb_loc, new_folder, new_folder, k), crop)
                k += 1

    def file_write(self,txt_file, file_name):
        loc = '/content/Final_Results/sorted_word_labels/'+file_name
        with open(loc, 'w') as f: 
            c=0
            for line in txt_file:  
                for l in line:
                    c+=1
                    if c == len(line):
                        f.write('%s' % l)
                    else:
                        f.write('%s ' % l)
                f.write("\n")
                c=0


In [ ]:
# Sorted word label location...
loc2 = '/content/Final_Results/sorted_word_labels/'
os.mkdir(loc2)

In [ ]:
loc = '/content/Final_Results/Final_word_segmentation_result/'
loc1 = '/content/Final_Results/Final_Line_Segmentation/'
obj = word_segmentation(loc, loc1)

Sorting the lines with word Bounding Box...

In [ ]:
filenm = "/content/Final_Results/Final_Word_BB/" 
os.mkdir(filenm)
img_path = "/content/Final_Results/Final_Line_Segmentation/" 
label_path = "/content/Final_Results/sorted_word_labels/" 
flag = 1
img_list = os.listdir(img_path)
lbl_list = os.listdir(label_path)
print(len(img_list))
print(len(lbl_list))
for i in img_list:
  temp = i.split('.')[0]
  txt_file = temp + '.txt'
  img_path1 = img_path + i
  label_path1 = label_path + txt_file
  img = draw_BB(img_path1, label_path1, flag)
  filenm1 = filenm + i
  cv2.imwrite(filenm1, img)


# **Saving Results** (If want to save the generated outputs: Please uncomment the last code section and assign directory of any folder of your drive to the defined 'final_save' variable.)

In [ ]:
# Copy content from one directory to another...
from_dir = "/content/yolov5/runs/detect"
to_dir = "/content/Final_Results/All_detect"
copy_dir(from_dir, to_dir)

In [ ]:
# Copy content from one directory to another...
from_dir = "/content/croped_line_after_1st_detection"
to_dir = "/content/Final_Results/Initial Line Segmentation"
copy_dir(from_dir, to_dir)

In [ ]:
zip_results()

To save the results, remove command from the cell below and put your desire drive path to the variable named "final_save" ,and run the below cell. Generated output is saved on this location as zip file: "/content/test_results/"



In [ ]:
# # Mounting Drive...
# from google.colab import drive
# drive.mount('/content/drive')

# final_save = "/content/drive/MyDrive/thesis/Transition_Outputs/"
# final_dir = "/content/test_results/"
# %cp -rip $final_dir $final_save
# print("Saved successfully!")

Or you can remove this below cells commants and run it to download the zip, although **we do not recommend this** as it might take more than couple of hours.

In [ ]:
# from google.colab import files
# files.download("/content/test_results/Final_Results.zip")